In [14]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [5]:
METADATA = "/dataset/meta/meta/"
IMGS = "/dataset/images/"

classes = pd.read_csv(METADATA+'classes.txt', header=None) # classes.txt contains the class names
labels = pd.read_csv(METADATA+'labels.txt', header=None) # labels.txt contains the image names

toNominal = dict(zip(classes[0].values, range(classes.shape[0]))) # class to norminal

# setup for model and data splits

# training data
train_df = pd.read_csv(METADATA+'train.txt', names=['txt'], header=None) 
train_df['img'] = train_df['txt'].apply(lambda x : x+'.jpg')
train_df['label'] = train_df['txt'].apply(lambda x: toNominal[x.split('/')[0]])
train_df.drop(['txt'], axis=1, inplace=True)
train_df = train_df.sample(frac=1) 

# validation data
valid_df = pd.read_csv(METADATA+'test.txt', names=['txt'], header=None) # 
valid_df['img'] = valid_df['txt'].apply(lambda x: x+'.jpg') 
valid_df['label'] = valid_df['txt'].apply(lambda x: toNominal[x.split('/')[0]])
valid_df.drop(['txt'], axis=1, inplace=True)    
valid_df = valid_df.sample(frac=1)
         
# shape of the data
print(f'train_data: {train_df.shape}') # (75750, 2)
print(f'valid_data: {valid_df.shape}') # (25250, 2)

FileNotFoundError: [Errno 2] No such file or directory: '/dataset/food41/meta/meta/classes.txt'

In [6]:
# augmentation of the data to create a uniform environment of data for the model

train_dg = ImageDataGenerator(
        rescale=1./255,
        rotation_range=30,
        shear_range=0.3,
        horizontal_flip=True,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.25,
)

valid_dg = ImageDataGenerator(
        rescale=1./255,
)

In [9]:
# setup dataset
BATSIZE = 64
IMSIZE = 224

train_data = train_dg.flow_from_dataframe(
    dataframe=train_df, # dataframe with image names and labels
    directory=IMGS, 
    x_col="img",
    y_col="label", 
    target_size=(IMSIZE, IMSIZE), # size of the images
    BATSIZE=BATSIZE,    
    class_mode="raw",
    shuffle=True, # shuffels the images and labes in the batch
)
valid_data = valid_dg.flow_from_dataframe(
    dataframe=valid_df,
    directory=IMGS,
    x_col="img",
    y_col="label",
    target_size=(IMSIZE, IMSIZE),
    BATSIZE=BATSIZE,
    class_mode="raw",
    shuffle=True,
)

NameError: name 'train_df' is not defined

In [10]:
x, y = next(train_data)
fig, ax = plt.subplots(nrows=5, ncols=5, figsize=(15,15))

for i in range(5):
    for j in range(5):
        ax[i][j].imshow(x[i+j*5])
        ax[i][j].set_title(labels[0][y[i+j*5]])
        ax[i][j].set_xticks([])
        ax[i][j].set_yticks([])
        
fig.show()

NameError: name 'train_data' is not defined

In [11]:
pre_trained = keras.applications.MobileNetV3Large(input_shape=(IMAGE_SIZE,IMAGE_SIZE,3),include_top=False,weights='imagenet')   # pre-trained model
pre_trained.trainable = True # trainable model

12683000/12683000 [==============================] - 3s 0us/step


In [12]:
model = keras.Sequential([  # define the laters in the CNN
    pre_trained,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(101, activation='softmax'),
])

model.compile(  # compiles the CNN network
    optimizer='adam',
    loss = 'sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 MobilenetV3large (Functiona  (None, 7, 7, 960)        2996352   
 l)                                                              
                                                                 
 global_average_pooling2d (G  (None, 960)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 128)               123008    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 101)               13029     
                                                                 
Total params: 3,132,389
Trainable params: 3,107,989
Non-

In [15]:
EPOCHS=20  # training epochs

reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor = 'val_accuracy',patience = 1,verbose = 1) # reduce learning rate if the model does not improve
early_stop = keras.callbacks.EarlyStopping(monitor = 'val_accuracy',patience = 5,verbose = 1,restore_best_weights = True) # early stopping
chkp = keras.callbacks.ModelCheckpoint('mobilenet_v3_large_checkpoint.h5',monitor='val_accuracy',verbose=1,save_best_only=True) # saves the model after every epoch

with tf.device('/GPU:0'):   # use GPU for training (I LOVE NVIDIA CUDA)
    hist = model.fit(   # fits the model to the data for each epoch using the GPU
        train_data,
        validation_data = valid_data,
        epochs = EPOCHS,
        callbacks=[early_stop, reduce_lr, chkp],
    )
    
model.save('ModelCNN.h5')   # saves the model after training to an .h5 file

NameError: name 'train_data' is not defined

In [ ]:
# Accuracy around 81.5% after 20 epochs